In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats 
# Import SymPy: 
from sympy import * 

## SYMPY

In [15]:
# Defining the parameters:
mu1   =  3.5
sig1  =  0.4
mu2   =  0.8
sig2  =  0.2
rho12 =  0.5           # Correlation parameter!

In [16]:
from IPython.core.display import Latex

def lprint(*args,**kwargs):
    """Pretty print arguments as LaTeX using IPython display system 
    
    Parameters
    ----------
    args : tuple 
        What to print (in LaTeX math mode)
    kwargs : dict 
        optional keywords to pass to `display` 
    """
    display(Latex('$$'+' '.join(args)+'$$'),**kwargs)
    
from IPython.core.display import Latex

def lprint(*args,**kwargs):
    """Pretty print arguments as LaTeX using IPython display system 
    
    Parameters
    ----------
    args : tuple 
        What to print (in LaTeX math mode)
    kwargs : dict 
        optional keywords to pass to `display` 
    """
    display(Latex('$$'+' '.join(args)+'$$'),**kwargs)
    
def myDiff(formula):
    return sqrt((formula.diff(L) * dL)**2 + (formula.diff(W) * dW)**2)

def myDiffWithCorr(formula, name = "", printNow = False):
    dd = sqrt((formula.diff(L) * dL)**2 + (formula.diff(W) * dW)**2 + 2*(formula.diff(L)*formula.diff(W)*(sigCorr**2)))
    if(printNow):
        lprint(latex(Eq(symbols('sigma_'+name), dd)))
    fd = lambdify((L,dL,W,dW,sigCorr),dd)
    return dd, fd
    
def diff_and_print(formula, name = ""):
    # Calculate uncertainty and print original relation/formula and the uncertainty
    dd = myDiff(formula)
    lprint(latex(Eq(symbols(name),formula)))
    lprint(latex(Eq(symbols('sigma_'+name), dd)))
    
def lambdifyFormula(formula, *args, name = ""):
    # Turn expression into numerical functions 
    f = lambdify((L,W),formula)
    d = myDiff(formula)
    fd = lambdify((L,dL,W,dW),d)
    return f, fd


In [18]:
# Define variables:
L,W,P = symbols("L, W, P")
dL,dW,dP = symbols("sigma_L, sigma_W, sigma_P")

# Define functions:

P = 2*L + 2*W

#print
diff_and_print(P,"P")

#get derivative 
dP = myDiff(P)

# Turn expressions into numerical functions 
fP, fdP = lambdifyFormula(P,"P")

# Define values and their errors
vL, vdL = mu1,sig1
vW, vdW = mu2,sig2

# Numerically evaluate expressions and print 
vP = fP(vL,vW)
vdP = fdP(vL,vdL,vW,vdW)

lprint(fr'P = ({vP:.1f} \pm {vdP:.1f})\,\mathrm{{m}}')

#Adding correlations (and also derivation, printing and lambdifying)
sigCorr = symbols("sigma_LW")
rho = symbols("rho_LW")

# Calculate uncertainty and print:
dP = sqrt((P.diff(L) * dL)**2 + (P.diff(W) * dW)**2)
lprint(latex(Eq(symbols('sigma_P'), dP)))

# Turn expression into numerical functions 
fP = lambdify((L,W),P)
fdP = lambdify((L,dL,W,dW),dP)

#CORRELATIONS
# sCorr = sqrt(rho*dL*dW)
# fSC = lambdify((rho,dL,dW),sCorr)

# vSigmaCorr = fSC(rho12,vdL,vdW)

# # Numerically evaluate expressions and print 
# vdP = fdP(vL,vdL,vW,vdW,vSigmaCorr)

# Define values and their errors
vL, vdL = mu1,sig1
vW, vdW = mu2,sig2

# Numerically evaluate expressions and print 
vP = fP(vL,vW)
vdP = fdP(vL,vdL,vW,vdW)
lprint(fr'P = ({vP:.1f} \pm {vdP:.1f})\,\mathrm{{m}}')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## SIMULATION
if not stated otherwise errors are gaussians

In [21]:
mean_L = 3.5
std_L = 0.4
mean_W = 0.8
std_W = 0.2
num = 10000     #amount of random numbers 

#generate random numbers 
random_gaussian_L = stats.norm.rvs(loc=mean_L, scale=std_L,size=num) 
random_gaussian_W = stats.norm.rvs(loc=mean_W, scale=std_W,size=num) 

In [30]:
P_rand = 2*random_gaussian_L + 2*random_gaussian_W
P_std = P_rand.std()
P = P_rand.mean()
print(f'P=({P:.1f}+/-{P_std:.1f})')

P=(8.6+/-0.9)


## ERROR ON A FRACTION

In [25]:
def frac_error(a,b):
    """params:
       a: number 1
       b: number 2
       return: fraction with its error"""
    frac = a/b
    frac_error = np.sqrt((frac * (1-frac)) / b)
    return frac,frac_error


In [28]:
#example 
total = 103261
positives = 2464 
fraction, error = frac_error(positives,total)
print(f'{fraction:.4f}+/-{error:.4f}')

0.0239+/-0.0005


## WEIGHTED MEAN

In [60]:
def weighted_mean(values, uncertanties):
    """params:
       values: list of measurments
       uncertanties: list of uncertanties
       return: weighted mean and error"""
    weights= 1/uncertanties**2
    weighted_mean=np.average(values, weights=weights)
    error = np.sqrt(1/sum(weights))
#     variance = np.average((values-weighted_mean)**2,weights = weights)
#     weighted_std = np.sqrt(variance)
    return weighted_mean,error


In [61]:
#example 
results = np.array([ 10.02, 9.87, 9.98, 9.86, 9.86, 9.81, 9.79]) 
uncertanties= np.array([ 0.11, 0.08, 0.14, 0.06, 0.03, 0.13, 0.04]) 
results_mean, error = weighted_mean(results, uncertanties)
print(f'The best estimate  is {results_mean:.2f}+\-{error:.3f}')

The best estimate  is 9.85+\-0.021
